<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/225_Cross_Sell_Upsell_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routine analysis utilities for Tier 2 Cross-Sell & Upsell Orchestrator

In [ ]:
"""Routine analysis utilities for Tier 2 Cross-Sell & Upsell Orchestrator"""

from typing import Dict, List, Any, Set
from datetime import datetime, timedelta


def analyze_customer_routine(
    customer: Dict[str, Any],
    product_lookup: Dict[str, Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Analyze customer's current routine: extract products, categories, and routine steps.

    Args:
        customer: Customer data dict with products_owned and categories
        product_lookup: Dictionary mapping product_id to product dict

    Returns:
        Dictionary with:
        - customer_products: List of product_ids customer owns
        - customer_categories: List of categories customer has products in
        - customer_routine_steps: List of routine steps customer has (1-5)
    """
    customer_products = []
    customer_categories_set: Set[str] = set()
    customer_routine_steps_set: Set[int] = set()

    # Extract from products_owned
    for product_owned in customer.get("products_owned", []):
        product_id = product_owned.get("product_id")
        if product_id:
            customer_products.append(product_id)

            # Look up product details
            product = product_lookup.get(product_id)
            if product:
                # Add category
                category = product.get("category")
                if category:
                    customer_categories_set.add(category)

                # Add routine step
                routine_step = product.get("routine_step")
                if routine_step is not None:
                    customer_routine_steps_set.add(routine_step)

    # Also check customer's categories field (may have more categories)
    customer_categories_list = customer.get("categories", [])
    if customer_categories_list:
        customer_categories_set.update(customer_categories_list)

    return {
        "customer_products": customer_products,
        "customer_categories": sorted(list(customer_categories_set)),
        "customer_routine_steps": sorted(list(customer_routine_steps_set))
    }


def identify_routine_gaps(
    customer_categories: List[str],
    customer_routine_steps: List[int],
    essential_categories: List[str],
    routine_steps_total: int = 5
) -> Dict[str, Any]:
    """
    Identify routine gaps: missing essential categories and missing routine steps.

    Args:
        customer_categories: List of categories customer has
        customer_routine_steps: List of routine steps customer has (1-5)
        essential_categories: List of essential categories (e.g., ["cleanser", "moisturizer", "spf"])
        routine_steps_total: Total number of routine steps (default: 5)

    Returns:
        Dictionary with:
        - routine_gaps: List of missing essential categories
        - routine_step_gaps: List of missing routine steps (1-5)
    """
    # Find missing essential categories
    customer_categories_set = set(customer_categories)
    essential_categories_set = set(essential_categories)
    routine_gaps = sorted(list(essential_categories_set - customer_categories_set))

    # Find missing routine steps
    customer_routine_steps_set = set(customer_routine_steps)
    all_routine_steps = set(range(1, routine_steps_total + 1))
    routine_step_gaps = sorted(list(all_routine_steps - customer_routine_steps_set))

    return {
        "routine_gaps": routine_gaps,
        "routine_step_gaps": routine_step_gaps
    }


def check_replenishment_needs(
    customer: Dict[str, Any],
    product_lookup: Dict[str, Dict[str, Any]],
    replenishment_warning_days: int = 7
) -> List[Dict[str, Any]]:
    """
    Check which products need replenishment based on purchase date and cycle.

    Args:
        customer: Customer data dict with products_owned
        product_lookup: Dictionary mapping product_id to product dict
        replenishment_warning_days: Days before due date to flag as "approaching" (default: 7)

    Returns:
        List of replenishment needs, each with:
        - product_id: Product ID
        - days_since_purchase: Days since last purchase
        - replenishment_cycle_days: Expected cycle in days
        - replenishment_due: Boolean (True if days_since_purchase >= cycle)
        - approaching_replenishment: Boolean (True if within warning days)
    """
    replenishment_needs = []
    today = datetime.now().date()

    for product_owned in customer.get("products_owned", []):
        product_id = product_owned.get("product_id")
        purchase_date_str = product_owned.get("purchase_date")

        if not product_id or not purchase_date_str:
            continue

        # Look up product to get replenishment cycle
        product = product_lookup.get(product_id)
        if not product:
            continue

        replenishment_cycle_days = product.get("replenishment_cycle_days")
        if replenishment_cycle_days is None:
            continue

        # Parse purchase date
        try:
            purchase_date = datetime.strptime(purchase_date_str, "%Y-%m-%d").date()
        except (ValueError, TypeError):
            continue

        # Calculate days since purchase
        days_since_purchase = (today - purchase_date).days

        # Check if replenishment is due
        replenishment_due = days_since_purchase >= replenishment_cycle_days

        # Check if approaching replenishment (within warning days)
        days_until_due = replenishment_cycle_days - days_since_purchase
        approaching_replenishment = (
            not replenishment_due and
            days_until_due <= replenishment_warning_days and
            days_until_due > 0
        )

        replenishment_needs.append({
            "product_id": product_id,
            "days_since_purchase": days_since_purchase,
            "replenishment_cycle_days": replenishment_cycle_days,
            "replenishment_due": replenishment_due,
            "approaching_replenishment": approaching_replenishment,
            "purchase_date": purchase_date_str
        })

    return replenishment_needs



# Tests for routine analysis utilities

In [ ]:
"""Tests for routine analysis utilities"""

from agents.tier2_crosssell_upsell.utilities.routine_analysis import (
    analyze_customer_routine,
    identify_routine_gaps,
    check_replenishment_needs
)
from agents.tier2_crosssell_upsell.utilities.data_loading import (
    load_customer_data,
    load_product_catalog,
    build_product_lookup
)


def test_analyze_customer_routine():
    """Test analyze_customer_routine extracts routine correctly"""
    print("\n=== Testing analyze_customer_routine ===")

    # Load real data
    customer = load_customer_data("C001")
    assert customer is not None, "Should load customer C001"

    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Analyze routine
    result = analyze_customer_routine(customer, product_lookup)

    # Verify structure
    assert "customer_products" in result, "Should return customer_products"
    assert "customer_categories" in result, "Should return customer_categories"
    assert "customer_routine_steps" in result, "Should return customer_routine_steps"

    # Verify customer_products
    assert isinstance(result["customer_products"], list), "customer_products should be a list"
    assert len(result["customer_products"]) > 0, "Should have at least one product"

    # Verify customer_categories
    assert isinstance(result["customer_categories"], list), "customer_categories should be a list"
    assert len(result["customer_categories"]) > 0, "Should have at least one category"

    # Verify customer_routine_steps
    assert isinstance(result["customer_routine_steps"], list), "customer_routine_steps should be a list"
    assert all(1 <= step <= 5 for step in result["customer_routine_steps"]), "Steps should be 1-5"

    print("✅ analyze_customer_routine test passed!")
    print(f"   Products: {len(result['customer_products'])}")
    print(f"   Categories: {result['customer_categories']}")
    print(f"   Routine steps: {result['customer_routine_steps']}")

    return result


def test_identify_routine_gaps():
    """Test identify_routine_gaps finds missing categories and steps"""
    print("\n=== Testing identify_routine_gaps ===")

    # Test with customer who has all essentials (C001)
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    routine_analysis = analyze_customer_routine(customer, product_lookup)

    essential_categories = ["cleanser", "moisturizer", "spf"]
    gaps = identify_routine_gaps(
        routine_analysis["customer_categories"],
        routine_analysis["customer_routine_steps"],
        essential_categories,
        routine_steps_total=5
    )

    # Verify structure
    assert "routine_gaps" in gaps, "Should return routine_gaps"
    assert "routine_step_gaps" in gaps, "Should return routine_step_gaps"

    assert isinstance(gaps["routine_gaps"], list), "routine_gaps should be a list"
    assert isinstance(gaps["routine_step_gaps"], list), "routine_step_gaps should be a list"

    print("✅ identify_routine_gaps test passed!")
    print(f"   Routine gaps (missing categories): {gaps['routine_gaps']}")
    print(f"   Routine step gaps: {gaps['routine_step_gaps']}")

    # Test with customer missing essentials (C002)
    customer2 = load_customer_data("C002")
    if customer2:
        routine_analysis2 = analyze_customer_routine(customer2, product_lookup)
        gaps2 = identify_routine_gaps(
            routine_analysis2["customer_categories"],
            routine_analysis2["customer_routine_steps"],
            essential_categories,
            routine_steps_total=5
        )
        print(f"   C002 gaps: {gaps2['routine_gaps']}, step gaps: {gaps2['routine_step_gaps']}")

    return gaps


def test_check_replenishment_needs():
    """Test check_replenishment_needs calculates replenishment correctly"""
    print("\n=== Testing check_replenishment_needs ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Check replenishment needs
    replenishment_needs = check_replenishment_needs(customer, product_lookup)

    # Verify structure
    assert isinstance(replenishment_needs, list), "Should return a list"

    # Verify each need has required fields
    for need in replenishment_needs:
        assert "product_id" in need, "Each need should have product_id"
        assert "days_since_purchase" in need, "Each need should have days_since_purchase"
        assert "replenishment_cycle_days" in need, "Each need should have replenishment_cycle_days"
        assert "replenishment_due" in need, "Each need should have replenishment_due"
        assert "approaching_replenishment" in need, "Each need should have approaching_replenishment"

        assert isinstance(need["days_since_purchase"], int), "days_since_purchase should be int"
        assert isinstance(need["replenishment_due"], bool), "replenishment_due should be bool"
        assert isinstance(need["approaching_replenishment"], bool), "approaching_replenishment should be bool"

    print("✅ check_replenishment_needs test passed!")
    print(f"   Replenishment needs found: {len(replenishment_needs)}")

    # Show some examples
    for need in replenishment_needs[:3]:
        print(f"   Product {need['product_id']}: {need['days_since_purchase']} days since purchase, "
              f"cycle={need['replenishment_cycle_days']}, due={need['replenishment_due']}")

    return replenishment_needs


def test_routine_analysis_integration():
    """Test all routine analysis utilities together"""
    print("\n=== Testing routine analysis integration ===")

    # Load data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Step 1: Analyze routine
    routine_analysis = analyze_customer_routine(customer, product_lookup)

    # Step 2: Identify gaps
    essential_categories = ["cleanser", "moisturizer", "spf"]
    gaps = identify_routine_gaps(
        routine_analysis["customer_categories"],
        routine_analysis["customer_routine_steps"],
        essential_categories,
        routine_steps_total=5
    )

    # Step 3: Check replenishment
    replenishment_needs = check_replenishment_needs(customer, product_lookup)

    print("✅ Integration test passed!")
    print(f"   Customer: {customer['name']} ({customer['customer_id']})")
    print(f"   Products: {len(routine_analysis['customer_products'])}")
    print(f"   Categories: {routine_analysis['customer_categories']}")
    print(f"   Routine steps: {routine_analysis['customer_routine_steps']}")
    print(f"   Missing categories: {gaps['routine_gaps']}")
    print(f"   Missing steps: {gaps['routine_step_gaps']}")
    print(f"   Replenishment needs: {len(replenishment_needs)}")

    return {
        "routine_analysis": routine_analysis,
        "gaps": gaps,
        "replenishment_needs": replenishment_needs
    }


if __name__ == "__main__":
    """Run all tests"""
    print("=" * 60)
    print("Testing Tier 2 Routine Analysis Utilities")
    print("=" * 60)

    # Test individual utilities
    test_analyze_customer_routine()
    test_identify_routine_gaps()
    test_check_replenishment_needs()

    # Integration test
    test_routine_analysis_integration()

    print("\n" + "=" * 60)
    print("✅ All routine analysis utility tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_routine_analysis
============================================================
Testing Tier 2 Routine Analysis Utilities
============================================================

=== Testing analyze_customer_routine ===
✅ analyze_customer_routine test passed!
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]

=== Testing identify_routine_gaps ===
✅ identify_routine_gaps test passed!
   Routine gaps (missing categories): []
   Routine step gaps: [2]
   C002 gaps: ['moisturizer', 'spf'], step gaps: [4, 5]

=== Testing check_replenishment_needs ===
✅ check_replenishment_needs test passed!
   Replenishment needs found: 6
   Product P102: 23 days since purchase, cycle=30, due=False
   Product P105: 21 days since purchase, cycle=30, due=False
   Product P108: 18 days since purchase, cycle=30, due=False

=== Testing routine analysis integration ===
✅ Integration test passed!
   Customer: Sarah Lee (C001)
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

============================================================
✅ All routine analysis utility tests passed!
============================================================
